In [50]:
import pandas as pd
import numpy as np
movies = pd.read_csv('tmdb_3000_movies.csv')
mov=pd.read_csv('tmdb_3000_movies.csv')
df = pd.read_csv('tmdb_3000_credits.csv')
df.columns = ['id', 'tittle', 'cast', 'crew']
j=mov[['original_title']]
movies = movies.merge(df, on='id')
movies.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2991 entries, 0 to 2990
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                2991 non-null   int64  
 1   genres                2991 non-null   object 
 2   homepage              1174 non-null   object 
 3   id                    2991 non-null   int64  
 4   keywords              2991 non-null   object 
 5   original_language     2991 non-null   object 
 6   original_title        2991 non-null   object 
 7   overview              2990 non-null   object 
 8   popularity            2991 non-null   float64
 9   production_companies  2991 non-null   object 
 10  production_countries  2991 non-null   object 
 11  release_date          2991 non-null   object 
 12  revenue               2991 non-null   int64  
 13  runtime               2990 non-null   float64
 14  spoken_languages      2991 non-null   object 
 15  status               

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
# removing english stop word like a, and , the 
tfidf = TfidfVectorizer(analyzer = 'word',stop_words = 'english')
#NaN -> ‘’
movies['overview'] = movies['overview'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies['overview'])
tfidf_matrix.shape

(2991, 16077)

In [52]:
from sklearn.metrics.pairwise import linear_kernel
cosin_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [53]:
index_of_movies = pd.Series(movies.index,index=movies['title']).drop_duplicates()

In [54]:
def get_recommendations(title, cosin_sim=cosin_sim):
    idx = index_of_movies[title]
    
    sim_scores = list(enumerate(cosin_sim[idx]))
    # sorting of moviesidx based on similarity score
    sim_scores = sorted(sim_scores, key = lambda x:x[1], reverse = True)
    # get top 10 of sorted 
    sim_scores = sim_scores[1:31]
    
    movies_idx = [i[0] for i in sim_scores]
    
    return movies['title'].iloc[movies_idx]

In [55]:
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for f in features:
    movies[f] = movies[f].apply(literal_eval)
# to get director from job
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
# get top 3 elements of list
def get_list(x):
    if isinstance(x, list):
        names = [ i['name'] for i in x]
        
        if len(names)  > 3:
            names = names[:3]
        return names
    return []
#apply all functions
movies['director'] = movies['crew'].apply(get_director)
features = ['cast', 'keywords', 'genres']
for f in features:
    movies[f] = movies[f].apply(get_list)
#striping
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''
features = ['cast', 'keywords', 'director', 'genres']
for f in features:
    movies[f] = movies[f].apply(clean_data)
#creating a SOUP
def create_soup(x):
    return ' '.join(x['keywords'])+' '+' '.join(x['cast'])+' '+x['director']+' '+' '.join(x['genres'])
movies['soup'] = movies.apply(create_soup, axis=1)
#count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words = 'english')
count_matrix = count.fit_transform(movies['soup'])
# finding similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
cosin_sim2 = cosine_similarity(count_matrix, count_matrix)

In [56]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
ratings = pd.read_csv('ratings_small.csv')

In [57]:
data=Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
svd=SVD()
cross_validate(svd,data,measures=['RMSE'],cv=5,verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8973  0.9018  0.8895  0.9023  0.8946  0.8971  0.0047  
Fit time          4.60    4.59    4.44    4.48    4.45    4.51    0.07    
Test time         0.30    0.32    0.23    0.33    0.24    0.29    0.04    


{'test_rmse': array([0.89727705, 0.90178474, 0.88953493, 0.90229621, 0.89459222]),
 'fit_time': (4.59644627571106,
  4.5948486328125,
  4.443248271942139,
  4.479581356048584,
  4.453047037124634),
 'test_time': (0.3025524616241455,
  0.322676420211792,
  0.2320690155029297,
  0.32884716987609863,
  0.2429208755493164)}

In [58]:
train = data.build_full_trainset()
svd.fit(train)

In [59]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.7792921722964596, details={'was_impossible': False})

In [60]:
movie_id = pd.read_csv('links.csv')[['movieId', 'tmdbId']]
movie_id.columns = ['movieId','id']
movie_id = movie_id.merge(movies[['title', 'id']], on='id').set_index('title')
print(movie_id.shape)

(2970, 2)


In [61]:
index_map = movie_id.set_index('id')

In [62]:
def recommend_for(userId,title):
    index = index_of_movies[title]
    tmdbId = movie_id.loc[title]['id']
    

    #content based
    sim_scores = list(enumerate(cosin_sim2[int(index)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:30]
    movie_indices = [i[0] for i in sim_scores]

    mv = movies.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    mv = mv[mv['id'].isin(movie_id['id'])]
    # CF
    mv['est'] = mv['id'].apply(lambda x: svd.predict(userId, index_map.loc[x]['movieId']).est)
    k=mv.head(6)
    l=k.index.tolist()
    print(l)
    return mv.head(5)

In [67]:
u=input("Enter User ID : ")
n=input("Enter Movie Name : ")
n=n.capitalize()
print(n)
genre=["Fantasy","Action","Adventure","Science Fiction","Crime","Drama","Thriller","Animation","Family","Western","Romance","Comedy","Horror","Mystery","War","History","Music",]
if n in genre:
    v=mov["genres"]
    cou=2
    p=[]
    for i in v:
        i=str(i)
        if n in i:
            p.append(cou)
        if len(p)==5:
            break
        cou+=1
    print(p)
else:
    m=j[j['original_title'].str.contains(n)]
    m=m["original_title"].values[0]
    print(recommend_for(u,m))


Avatar
[206, 71, 103, 131, 215, 466]
                                      title  vote_count  vote_average     id  \
206                     Clash of the Titans        2233           5.6  18823   
71    The Mummy: Tomb of the Dragon Emperor        1387           5.2   1735   
103               The Sorcerer's Apprentice        1470           5.8  27022   
131                                 G-Force         510           5.1  19585   
215  Fantastic 4: Rise of the Silver Surfer        2589           5.4   1979   

          est  
206  3.255556  
71   3.410865  
103  3.668233  
131  3.409176  
215  3.003754  
